# MLE-Bench Demo

Two ways to evaluate AI agents:

**1. Competition Runs** – Full Kaggle competitions (build models, make predictions, score against leaderboard)

**2. Technique-Tasks** – Focused ML skill assessments: `imbalance`, `missing`, `encoding`, `cv`, `scaling`, `leakage`

**Competitions:** Edit `experiments/splits/dev.txt` to choose which competitions to run.  
Current: `random-acts-of-pizza`, `spooky-author-identification`, `dogs-vs-cats-redux-kernels-edition`

---
*First, read `RUN_DEMO.md` for setup instructions.*

## Setup

First, let's import the SDK and check the server is running.

In [1]:
import sys
sys.path.insert(0, '.')

from sdk import Client

client = Client()
health = client.health()
print(f"Server status: {health}")

/Users/sanketshah/Desktop/mle-bench-ss/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Server status: {'status': 'ok'}


---
# Option A: Competition Run (Original MLE-Bench)

Run the agent on a full Kaggle competition. The agent will:
1. Read the competition description
2. Explore and analyze the data
3. Build and train models
4. Generate predictions
5. Submit for scoring against the leaderboard

**Uncomment the cell below to run a competition instead of technique-tasks.**

In [ ]:
# OPTION A: Competition Run (uncomment to use)
# run_id = client.run(
#     agent_id="aide/dev",
#     competition_set="experiments/splits/dev.txt",
#     lite=True,
#     technique_tasks=False,  # Regular competition run
# )
# print(f"Started competition run: {run_id}")
# final_status = client.wait_for_completion(run_id, poll_interval=10, timeout=600)
# print(f"Final status: {final_status['status']}")

Started run: ac9f1374-2d14-4165-8f51-1cf8d97b3026
Status: running


---
# Option B: Technique-Tasks (ML Skill Assessment)

Run the agent on focused skill assessments. The agent will:
1. Read the technique-specific prompt (e.g., "analyze class imbalance")
2. Use the same competition data
3. Produce a focused analysis
4. Get graded on that specific skill

Available tasks: `imbalance`, `missing`, `encoding`, `cv`, `scaling`, `leakage`

**Alternate model**: To try an older agent preconfigured with more internal search steps, set `agent_id="aide/gpt-4-turbo-dev"` in the notebook — that agent is typically configured to use 8 steps.

In [ ]:
# OPTION B: Technique-Tasks Run (using this for the demo)
run_id = client.run(
    agent_id="aide/dev",
    competition_set="experiments/splits/dev.txt",
    lite=True,
    technique_tasks=True,   # Enable technique-task mode
    tasks=["imbalance"],    # Which tasks to run
)
print(f"Started run: {run_id}")

# Wait for completion (polls every 10s)
final_status = client.wait_for_completion(run_id, poll_interval=10, timeout=600)
print(f"Final status: {final_status['status']}")

## Review Results

Check what the agent produced and how it was graded.

In [14]:
run = client.get_run(run_id)
print(f"Run ID: {run['run_id']}")
print(f"Status: {run['status']}")
print(f"Run Group: {run.get('run_group')}")
print(f"Run Dir: {run.get('run_dir')}")

if run.get('logs'):
    print(f"\nLogs ({len(run['logs'])} entries):")
    for log in run['logs'][-3:]:
        print(f"  {log}")

Run ID: ac9f1374-2d14-4165-8f51-1cf8d97b3026
Status: completed
Run Group: 2025-12-11T02-42-45-UTC_run-group_aide
Run Dir: runs/2025-12-11T02-42-45-UTC_run-group_aide

Logs (3 entries):
  {'cmd': ['/Users/sanketshah/miniforge3/bin/python', '-m', 'mlebench.cli', 'prepare', '--list', 'experiments/splits/dev.txt', '--lite'], 'returncode': 0, 'stdout': "Warning: Looks like you're using an outdated API Version, please consider updating (server 1.8.0 / client 1.6.17)\nrandom-acts-of-pizza.zip: Skipping, found more recently modified local copy (use --force to force download)\nWarning: Looks like you're using an outdated API Version, please consider updating (server 1.8.0 / client 1.6.17)\nspooky-author-identification.zip: Skipping, found more recently modified local copy (use --force to force download)\nWarning: Looks like you're using an outdated API Version, please consider updating (server 1.8.0 / client 1.6.17)\ndogs-vs-cats-redux-kernels-edition.zip: Skipping, found more recently modified

## Browse Rollout Artifacts

View all files generated by the agent during the run.

In [ ]:
from pathlib import Path
import json

def show_tree(path, prefix="", max_depth=3, current_depth=0):
    """Display directory tree structure."""
    if current_depth >= max_depth:
        return
    path = Path(path)
    items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name))
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        connector = "└── " if is_last else "├── "
        print(f"{prefix}{connector}{item.name}")
        if item.is_dir():
            extension = "    " if is_last else "│   "
            show_tree(item, prefix + extension, max_depth, current_depth + 1)

if run.get('run_dir'):
    print(f"📂 {run['run_dir']}\n")
    show_tree(run['run_dir'])

## Inspect the Outputs

Let's look at the grading results and run artifacts.

In [ ]:
from pathlib import Path
import json

if run.get('run_dir'):
    run_dir = Path(run['run_dir'])
    print(f"Run directory: {run_dir}")
    
    # Check for technique grades (technique-tasks mode)
    grades_file = run_dir / "technique_grades.json"
    if grades_file.exists():
        grades = json.loads(grades_file.read_text())
        print(f"\n📊 Technique Grades:")
        print(json.dumps(grades, indent=2))
    
    # Check for competition grades (regular mode)
    report_file = run_dir / "grading_report.json"
    if report_file.exists():
        report = json.loads(report_file.read_text())
        print(f"\n🏆 Competition Results:")
        print(json.dumps(report, indent=2))
    
    # List contents
    print(f"\n📁 Run Contents:")
    for item in sorted(run_dir.iterdir()):
        print(f"  {item.name}")
else:
    print("No run directory found yet")

Run directory: runs/2025-12-11T02-42-45-UTC_run-group_aide

Technique Grades:
{
  "dogs-vs-cats-redux-kernels-edition": {
    "tasks": {
      "imbalance": {
        "task": "imbalance",
        "score": 0,
        "passed": false,
        "word_count": 0,
        "message": "imbalance_analysis.json not found"
      }
    },
    "summary": {
      "total": 1,
      "passed": 0,
      "total_words": 0,
      "pass_rate": 0.0
    }
  },
  "spooky-author-identification": {
    "tasks": {
      "imbalance": {
        "task": "imbalance",
        "score": 0,
        "passed": false,
        "word_count": 0,
        "message": "imbalance_analysis.json not found"
      }
    },
    "summary": {
      "total": 1,
      "passed": 0,
      "total_words": 0,
      "pass_rate": 0.0
    }
  },
  "random-acts-of-pizza": {
    "tasks": {
      "imbalance": {
        "task": "imbalance",
        "score": 0,
        "passed": false,
        "word_count": 0,
        "message": "imbalance_analysis.json n

## Agent Decision Trace (AIDE Journal)

AIDE logs its reasoning and code at each step. Useful for debugging agent behavior.

In [ ]:
if run.get('run_dir'):
    run_dir = Path(run['run_dir'])
    for comp_dir in run_dir.iterdir():
        if not comp_dir.is_dir():
            continue
        journal_file = comp_dir / "submission" / "journal.json"
        if journal_file.exists():
            print(f"📓 {journal_file}\n")
            journal = json.loads(journal_file.read_text())
            print(f"AIDE took {len(journal)} steps:")
            for i, step in enumerate(journal[:3]):
                print(f"\n--- Step {i+1} ---")
                if 'plan' in step:
                    print(f"Plan: {step['plan'][:200]}...")
                if 'code' in step:
                    print(f"Code: {len(step['code'])} chars")
            break
    else:
        print("No journal.json found")

---
## Next Steps

- **Try different tasks**: Change `tasks=["imbalance"]` to `["missing", "encoding", "cv"]`
- **Run full competitions**: Comment out Option B and uncomment Option A
- **Use more steps**: Change `aide/dev` to `aide` for more thorough agent exploration
- **Add more competitions**: Edit `experiments/splits/dev.txt` to include more

See `README.md` and `RUN_DEMO.md` for more details.